<a href="https://colab.research.google.com/github/JamesMartinOU/PublicRedditSentimentAnalysis/blob/main/RedditCommentsSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Python libraries
!pip install transformers torch emoji
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 34.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
# Import Python libraries
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import emoji
import pandas as pd
import mysql.connector
import warnings
from google.colab import files
import time
import openpyxl
from google.colab import auth
import gspread
from google.colab import files
from google.auth import default
from sqlalchemy import create_engine

In [ ]:
DB_HOST = "redditdb.cfk6syyo42ze.us-east-2.rds.amazonaws.com"
DB_USER = "JamesMartinOU"  # Your MySQL username
DB_PASSWORD = "Bedard99*"  # Your MySQL password
DB_NAME = "redditdb"  # Your database name

In [ ]:
# Function preprocess - removes emojies and cleans text input
def preprocess(text):
    return emoji.demojize(text).replace('\n', ' ').strip()

In [ ]:
# Function split_text - segments an input to meet tokenizer limit
def split_text(text, chunk_size=450):
    """
    Splits a long string into chunks of `chunk_size` characters.
    """
    text = preprocess(text)
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

In [ ]:
# Load Twitter sentiment model
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# MySQL connection
conn = mysql.connector.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME
)
cursor = conn.cursor()

# Import all data from reddit_comments
cursor.execute("SELECT * FROM reddit_comments;")
columns = [desc[0] for desc in cursor.description]
data = cursor.fetchall()

# Write reddit_comments query result to df
df = pd.DataFrame(data, columns=columns)
# Segment just the first 100000 comments
df = df.iloc[499999:600000].reset_index(drop=True)

# Close connection
cursor.close()
conn.close()

# Create results to store sentiment data
results = []

for idx, row in df.iterrows():
    post_id = row['post_id']
    comment_id = row['comment_id']
    body = row['body']

    if pd.isna(body):
        continue

    chunks = split_text(body)

    for i, chunk in enumerate(chunks):
        try:
            result = sentiment_pipeline(chunk, truncation=True, max_length=512)[0]
            label_map = {
                'LABEL_0': 'Negative',
                'LABEL_1': 'Neutral',
                'LABEL_2': 'Positive'
            }
            sentiment = label_map[result['label']]
            score = round(result['score'], 4)
            results.append({
                'post_id': post_id,
                'comment_id': comment_id,
                'segment': i + 1,
                'body_segment': chunk,
                'sentiment': sentiment,
                'confidence': score
            })
        except Exception as e:
            print(f"Error processing comment {comment_id} segment {i+1}: {e}")

# Convert to DataFrame
sentiment_df = pd.DataFrame(results)
# Save dataframe locally
sentiment_df.to_csv("reddit_sentiment_results.csv", index=False)
files.download("reddit_sentiment_results.csv")
# Preview
print(sentiment_df.head())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Device set to use cpu


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

   post_id comment_id  segment  \
0  10nczjx    j69rt1h        1   
1  10nczjx    j69rtsw        1   
2  10nczjx    j69rtz9        1   
3  10nczjx    j69rucn        1   
4  10nbau4    j69rwbw        1   

                                        body_segment sentiment  confidence  
0                                       Supersize Me   Neutral      0.7779  
1  Market pays what it needs to pay  If they cant...  Negative      0.7331  
2  So that's a fairly large caveat saying McDonal...   Neutral      0.6132  
3  Fuck those jobs then. Excited society doesn't ...  Negative      0.8726  
4  Media members aren't allowed to place bets on ...  Negative      0.7382  


In [ ]:
# Write sentiment_df to reddit_comments_sentiment table
db_url = f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"
engine = create_engine(db_url)
sentiment_df.to_sql(name='reddit_comments_sentiment', con=engine, if_exists='replace', index=False)
print("Sentiment results written to 'reddit_comments_sentiment' table.")

Sentiment results written to 'reddit_comments_sentiment' table.


In [ ]:
# Query result set
db_url = f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"
engine = create_engine(db_url)
# Run query and load into DataFrame
query = "SELECT * FROM reddit_comments_sentiment;"
df_sentiment = pd.read_sql(query, engine)
# Preview result
print(df_sentiment.head())

   post_id comment_id  segment  \
0  10nczjx    j69rt1h        1   
1  10nczjx    j69rtsw        1   
2  10nczjx    j69rtz9        1   
3  10nczjx    j69rucn        1   
4  10nbau4    j69rwbw        1   

                                        body_segment sentiment  confidence  
0                                       Supersize Me   Neutral      0.7779  
1  Market pays what it needs to pay  If they cant...  Negative      0.7331  
2  So that's a fairly large caveat saying McDonal...   Neutral      0.6132  
3  Fuck those jobs then. Excited society doesn't ...  Negative      0.8726  
4  Media members aren't allowed to place bets on ...  Negative      0.7382  
